In [ ]:
# Install dependencies
!pip3 install -U spacy contractions beautifulsoup4 spacy-cleaner pyspellchecker requests pandas scikit-learn tqdm matplotlib seaborn keras tensorflow
!python3 -m spacy download en_core_web_lg

In [ ]:
# Import statements
import re
from bs4 import BeautifulSoup
import contractions
from spellchecker import SpellChecker
import en_core_web_lg
import spacy
import requests
from tqdm.contrib.concurrent import process_map

import time
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.naive_bayes import MultinomialNB
from keras.utils import to_categorical
from keras import models
from keras import layers

In [ ]:
# Exploratory data analysis
imdb_data = pd.read_csv('input/imdb_dataset.csv')
print(imdb_data.head(10))
imdb_data.describe()

In [ ]:
def normalize_review(review: str) -> str:
    '''Method to normalize (preprocess and lemmatize) a review into a tokenized list of words
    Steps:
      1. HTML decoding
      2. Lowercase conversion
      3. Contractions expansion
      4. Digits and punctuations removal
      5. Word tokenization
      6. Stop words removal and removal of insignificant words (less than 2 characters)
      7. Lemmatization

    Required Args
      review (str): Text to be preprocessed

  Returns
      str: Tokenized text
  '''
  # HTML decoding
  review = BeautifulSoup(review).get_text()
  # Convert words to lowercase
  review = review.lower()
  # Expand contractions
  review_list = []
  for word in review.split():
      review_list.append(contractions.fix(word))
  review = ' '.join(review_list)
  # Digits and punctuations removal
  review_list = re.sub('[^a-zA-Z0-9_]', ' ', review).split()
  review_list = [word for word in review_list if not re.search(r'\d', word)]
  review = ' '.join(review_list)
  # Word tokenization, stop word removal and lemmatization
  nlp = en_core_web_lg.load()
  doc = nlp(review)
  tokenized_word_list = []
  for token in doc:
      if '_' in token.text:
          tokenized_word_list.append(token.text)
      elif not token.is_stop and len(token.text) > 1:
          # Lemmatization
          tokenized_word_list.append(token.lemma_)
  return ' '.join(tokenized_word_list)